In [1]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import random
import openpyxl
from openpyxl.chart import BarChart, Reference, PieChart
from openpyxl.drawing.image import Image
import matplotlib.pyplot as plt

# Step 1: Generate Mock LV Client Data 
def generate_mock_data(num_records=500):
    # Sample names & categories 
    sample_names = [
        "John Doe", "Jane Smith", "Alice Johnson", "Bob Brown", "Charlie Davis",
        "David Evans", "Emma Franklin", "Frank Green", "Grace Harris", "Henry Irving",
    ] * 50  
   
    categories = ['Handbags', 'Jewelry', 'Clothing', 'Shoes', 'Accessories']  # LV product categories
    today = datetime.now()
    data = []
   
    # Generate unique customers (approx num_records / avg_purchases ~100 customers)
    for i in range(100): 
        customer_id = f"CUST-{i:04d}"
        name = random.choice(sample_names)
        email = name.lower().replace(" ", ".") + "@example.com"
        loyalty_points = random.randint(0, 5000)
        num_purchases = random.randint(1, 10)
       
        for _ in range(num_purchases):
            # Bias toward more recent purchases so that some meet 90-day criteria
            days_ago = random.randint(1, 90) if random.random() > 0.3 else random.randint(91, 365)
            purchase_date = today - timedelta(days=days_ago)
            product_category = random.choice(categories)
            amount_spent = random.uniform(500, 5000)  # Luxury price range
           
            data.append({
                'CustomerID': customer_id,
                'Name': name,
                'Email': email,
                'PurchaseDate': purchase_date,
                'ProductCategory': product_category,
                'AmountSpent': amount_spent,
                'LoyaltyPoints': loyalty_points
            })
   
    df = pd.DataFrame(data)
    return df

# Step 2: Store in SQLite and Query
def setup_and_query_db(df):
    conn = sqlite3.connect('lv_clients.db')  # Create local DB
    df.to_sql('Purchases', conn, if_exists='replace', index=False)
   
    # SQL Query: Get aggregated data for high-value clients (e.g., total spend > 2000, recent purchases)
    query = """
    SELECT
        CustomerID,
        Name,
        Email,
        COUNT(*) AS Frequency,
        SUM(AmountSpent) AS TotalSpent,
        MAX(PurchaseDate) AS LastPurchase,
        LoyaltyPoints
    FROM Purchases
    GROUP BY CustomerID
    HAVING TotalSpent > 2000 AND LastPurchase > DATE('now', '-90 days')  -- High-value and recent
    ORDER BY TotalSpent DESC
    """
    results_df = pd.read_sql_query(query, conn)
    conn.close()
    return results_df

# Step 3: Analyze with Pandas (RFM Scoring)
def perform_rfm_analysis(df):
    if df.empty:
        print("Warning: No qualifying client data found based on criteria.")
        return df  
   
    today = datetime.now()
    df['Recency'] = (today - pd.to_datetime(df['LastPurchase'])).dt.days
    df['Frequency'] = df['Frequency']
    df['Monetary'] = df['TotalSpent']
   
    # Simple RFM Scoring (1-5 scale, higher = better)
    df['R_Score'] = pd.qcut(df['Recency'], 5, labels=[5, 4, 3, 2, 1], duplicates='drop')  # Low recency = high score
    df['F_Score'] = pd.qcut(df['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
    df['M_Score'] = pd.qcut(df['Monetary'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
    df['RFM_Score'] = df['R_Score'].astype(int) + df['F_Score'].astype(int) + df['M_Score'].astype(int)
   
    # Segment: e.g., 'Champions' for high RFM
    df['Segment'] = 'Standard'
    df.loc[df['RFM_Score'] > 12, 'Segment'] = 'Champions'  # Top clients for VIP campaigns
    df.loc[(df['RFM_Score'] > 8) & (df['RFM_Score'] <= 12), 'Segment'] = 'Loyal'
   
    return df

# Step 4: Generate Excel Report with Visuals
def generate_excel_report(df):
    output_file = 'lv_client_report.xlsx'
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Sheet 1: Raw Insights Data
        df.to_excel(writer, sheet_name='Client Insights', index=False)
       
        # Sheet 2: Summary Stats
        summary = df.groupby('Segment').agg({
            'CustomerID': 'count',
            'TotalSpent': 'sum',
            'Frequency': 'mean',
            'Recency': 'mean'
        }).rename(columns={'CustomerID': 'Count', 'Frequency': 'Avg Frequency', 'Recency': 'Avg Recency (Days)'})
        summary.to_excel(writer, sheet_name='Summary')
   
    # Add Charts using openpyxl 
    if not df.empty:
        wb = openpyxl.load_workbook(output_file)
        ws = wb['Summary']
   
        # Bar Chart: Total Spend by Segment
        bar_chart = BarChart()
        bar_chart.type = "col"
        bar_chart.title = "Total Spend by Client Segment"
        data = Reference(ws, min_col=3, min_row=1, max_row=ws.max_row, max_col=3)  # TotalSpent column
        cats = Reference(ws, min_col=1, min_row=2, max_row=ws.max_row)  # Segments
        bar_chart.add_data(data, titles_from_data=True)
        bar_chart.set_categories(cats)
        ws.add_chart(bar_chart, "F2")
   
        # Pie Chart: Client Distribution by Segment
        pie_chart = PieChart()
        pie_chart.title = "Client Distribution by Segment"
        pie_data = Reference(ws, min_col=2, min_row=2, max_row=ws.max_row)  # Count column (excluding header)
        pie_cats = Reference(ws, min_col=1, min_row=2, max_row=ws.max_row)  # Segments
        pie_chart.add_data(pie_data)
        pie_chart.set_categories(pie_cats)
        ws.add_chart(pie_chart, "F17")  
   
        # Matplotlib Scatter Plot: Recency vs Monetary, colored by RFM Score
        fig, ax = plt.subplots(figsize=(6, 4))
        scatter = ax.scatter(df['Recency'], df['Monetary'], c=df['RFM_Score'], cmap='viridis')
        ax.set_title('RFM Scatter: Recency vs Monetary Value')
        ax.set_xlabel('Recency (Days)')
        ax.set_ylabel('Monetary Value ($)')
        plt.colorbar(scatter, label='RFM Score')
        plt.savefig('rfm_scatter.png', dpi=100)
        plt.close()
   
        # Embed the PNG in Excel
        img = Image('rfm_scatter.png')
        ws.add_image(img, 'P2')  
   
        wb.save(output_file)
    print(f"Report generated: {output_file}")

# Main Automation Flow
if __name__ == "__main__":
    print("Generating mock LV client data...")
    mock_df = generate_mock_data()
   
    print("Setting up SQL DB and querying...")
    queried_df = setup_and_query_db(mock_df)
   
    print("Performing RFM analysis...")
    analyzed_df = perform_rfm_analysis(queried_df)
   
    print("Generating Excel report with enhanced visuals...")
    generate_excel_report(analyzed_df)
   
    print("Automation complete! Open 'lv_client_report.xlsx' in Microsoft Excel to view tables and visuals.")

Generating mock LV client data...
Setting up SQL DB and querying...
Performing RFM analysis...
Generating Excel report with enhanced visuals...
Report generated: lv_client_report.xlsx
Automation complete! Open 'lv_client_report.xlsx' in Microsoft Excel to view tables and visuals.
